In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604532,32.3,1292165,69.1,1292165,69.1
Vcells,1086868,8.3,8388608,64.0,1632674,12.5


In [2]:
semilla <- 100019
set.seed(semilla)

In [3]:
#Optimizacion Bayesiana de hiperparametros de  rpart
#funciona automaticamente con EXPERIMENTOS
#va generando incrementalmente salidas para kaggle

library("data.table")
library("rlist")
library("yaml")

library("ranger")
library("parallel")

#paquetes necesarios para la Bayesian Optimization
library("DiceKriging")
library("mlrMBO")

library(stringr)


Loading required package: mlr

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Loading required package: checkmate


Attaching package: 'checkmate'


The following object is masked from 'package:DiceKriging':

    checkNames




In [4]:
#para poder usarlo en la PC y en la nube
switch ( Sys.info()[['sysname']],
         Windows = { directory.root   <-  "M:\\" },   #Microsoft Windows
         Darwin  = { directory.root   <-  "~/dm/" },  #Apple MAC
#         Linux   = { directory.root   <-  "~/buckets/b1/" }  #Entorno Google Cloud
         Linux   = { directory.root   <-  "~/" }  #Hay un problema en el almacenamiento en cloud del BO por el momento se graba en el usuario
       )
#defino la carpeta donde trabajo

#Hay un error en el almacenamiento 

#setwd( directory.root )

In [5]:
kexperimento  <- NA   #NA si se corre la primera vez, un valor concreto si es para continuar procesando

kscript           <- "BO Ranger"
kBO_iter    <-  150   #cantidad de iteraciones de la Optimizacion Bayesiana

#kcarpeta_datasets    <- "./buckets/b1/datasets/"                          #VM o Ubuntu

setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
kcarpeta_datasets    <- "./datasets/"                          #VM o Ubuntu


#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022_FE_DC2.csv")


#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000


hs  <- makeParamSet(
          makeNumericParam("alpha"       , lower= -1   , upper=    0.1),
          makeIntegerParam("num.trees" , lower=  100L  , upper=   1500L),
          makeIntegerParam("min.node.size" , lower=  5L  , upper=   1000L),
          makeIntegerParam("max.depth" , lower=  3L  , upper=   20L),
          makeIntegerParam("mtry" , lower=  5L  , upper=   100L)
        )

ksemilla_azar  <- semilla

#------------------------------------------------------------------------------
#Funcion que lleva el registro de los experimentos

get_experimento  <- function()
{
  if( !file.exists( "./maestro.yaml" ) )  cat( file="./maestro.yaml", "experimento: 1000" )

  exp  <- read_yaml( "./maestro.yaml" )
  experimento_actual  <- exp$experimento

  exp$experimento  <- as.integer(exp$experimento + 1)
  Sys.chmod( "./maestro.yaml", mode = "0644", use_umask = TRUE)
  write_yaml( exp, "./maestro.yaml" )
  Sys.chmod( "./maestro.yaml", mode = "0444", use_umask = TRUE) #dejo el archivo readonly

  return( experimento_actual )
}

In [6]:
#------------------------------------------------------------------------------
#Funcion que lleva el registro de los experimentos

get_experimento  <- function()
{
  if( !file.exists( "./maestro.yaml" ) )  cat( file="./maestro.yaml", "experimento: 1000" )

  exp  <- read_yaml( "./maestro.yaml" )
  experimento_actual  <- exp$experimento

  exp$experimento  <- as.integer(exp$experimento + 1)
  Sys.chmod( "./maestro.yaml", mode = "0644", use_umask = TRUE)
  write_yaml( exp, "./maestro.yaml" )
  Sys.chmod( "./maestro.yaml", mode = "0444", use_umask = TRUE) #dejo el archivo readonly

  return( experimento_actual )
}

In [7]:
#------------------------------------------------------------------------------
#funcion para particionar, es la que Andres reemplaza con caret

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------

In [8]:
RFSimple  <- function( fold_test, data, param )
{
    formula_obj  <-  formula(paste("clase ~ ."))
    
    modelo  <- ranger(
                    data= data[ fold != fold_test, ],
                   formula_obj,  
                   probability=TRUE,
                    splitrule="gini",
                    alpha = param$alpha,
                    num.trees=param$num.trees,
                      min.node.size=param$min.node.size,
                  max.depth=param$max.depth,
                  mtry=param$mtry,
                    seed=semilla
                 ) 

dtest <- data[ fold==fold_test, ]
dtest$score <- predict(modelo, dtest, type = 'response')$predictions[,2]
dtest$Predicted <- dtest$score > 1/40
    

  ganancia_testing  <- 3*sum(dtest$Predicted*dtest[,clase]*kTPGain + 
                                     dtest$Predicted * (0 == dtest[,clase]) * kFPGain)

  return( ganancia_testing )
}

In [9]:
#------------------------------------------------------------------------------

RFCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )
  particionar( data, divi, seed=semilla, agrupa=pagrupa )

  ganancias  <- mcmapply( RFSimple, 
                          seq(qfolds), # 1 2 3 4 5  
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a 5 si posee Linux o Mac OS

  data[ , fold := NULL ]
  #devuelvo la primer ganancia y el promedio
  return( mean( unlist( ganancias )) *  qfolds )   #aqui normalizo la ganancia
}

In [10]:
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <-  RFCrossValidation( dtrain, param=x, qfolds= xval_folds, pagrupa="clase", semilla=ksemilla_azar )

   #si tengo una ganancia superadora, genero el archivo para Kaggle
   if(  ganancia > GLOBAL_ganancia_max )
   {
     GLOBAL_ganancia_max <<-  ganancia  #asigno la nueva maxima ganancia
    
     fwrite( x, 
             file= paste0(kkaggle, str_pad(GLOBAL_iteracion, 5, pad = "0"), "_params.txt" ),
             sep=  "," )
   write.table(ganancia, file = paste0(kkaggle, str_pad(GLOBAL_iteracion, 5, pad = "0"), "_ganancia.txt" ), sep = "\t",
            col.names = c("ganancia"), row.names=FALSE)
   }
    write.table(ganancia, file = paste0(kkaggle, str_pad(GLOBAL_iteracion, 5, pad = "0"), "_iteracion.txt" ), sep = "\t",
            col.names = c("iteracion_ganancia"), row.names=FALSE)
   return( ganancia )
}
#------------------------------------------------------------------------------

In [11]:
#Aqui empieza el programa

if( is.na(kexperimento ) )   kexperimento <- get_experimento()  #creo el experimento

In [12]:
#en estos archivos quedan los resultados
kbayesiana  <- paste0("./E",  kexperimento, "_rf.RDATA" )
kkaggle     <- paste0("./E",kexperimento, "_rf_kaggle_" )

In [13]:
GLOBAL_ganancia_max  <-  -Inf
GLOBAL_iteracion  <- 0

In [14]:
#cargo los datos
dataset <- fread(karchivo_entrada)

#Para hacer pruebas rapidas puedo reducir el dataset a una fraccion
#subsample <- sample(1:nrow(dataset), .1 * nrow(dataset))
#dataset <- dataset[subsample,]

dataset[is.na(dataset)] <- 0
dataset[,'clase' := as.integer(dataset[,'clase_ternaria'] == 'BAJA+2')]
dataset[ ,  'clase_ternaria' := NULL    ] 

dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar

dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [15]:
head(dataset)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,tarj_credito_mconsumototal,tarj_credito_cconsumos,tarj_credito_cadelantosefectivo,tarj_credito_mpagominimo,prop_limite_compra_saldo,prop_limite_compra_gastos,prop_limite_compra_ingresos,prop_limite_compra_saldo_estim,cant_prod_dif_desvinc,clase
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
31115668,202101,1,0,0,56,132,1897.12,24324.75,940.32,⋯,18435.59,1,0,1337.22,1.118721,1.804695,0.6512564,1.118721,4,0
31115678,202101,1,0,0,48,100,-384.72,33860.45,-515.53,⋯,0.00,0,0,0.00,0.000000,0.000000,0.0000000,0.000000,2,0
31117415,202101,1,0,0,60,53,-3622.66,-21310.32,4064.85,⋯,21182.80,6,0,20152.14,1.642494,1.780312,1.1214953,1.642494,2,0
31117711,202101,1,0,0,46,273,14542.05,125727.99,1037.77,⋯,758.21,3,0,1302.03,4.238328,14.654054,12.1098966,4.238328,9,0
31118145,202101,1,0,0,47,192,1868.27,37305.29,2861.62,⋯,6316.13,1,0,4680.27,14.582133,3.086385,16.7739531,14.582133,0,0
31118204,202101,1,0,0,68,258,2301.86,60781.98,5169.05,⋯,37036.92,17,0,4093.77,18.133433,4.933004,8.3994629,18.133433,23,0


In [16]:
#Aqui comienza la configuracion de la Bayesian Optimization

configureMlr( show.learner.output = FALSE)

funcion_optimizar  <- EstimarGanancia

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 60,  save.file.path= kbayesiana)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <-  makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

In [ ]:
#inicio la optimizacion bayesiana
if(!file.exists(kbayesiana)) {
  run  <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else  run  <- mboContinue( kbayesiana )   #retomo en caso que ya exista

Computing y column(s) for design. Not provided.



In [ ]:
quit( save="no" )